In [1]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score
from scipy import spatial

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/kuchuganova/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
cleaned_data = pd.read_csv('cleaned.csv')
cleaned_data = cleaned_data.dropna()


In [3]:
cleaned_data

,name_1,name_2,is_duplicate
0,iko industries ltd,enormous industrial trade pvt ltd,0
1,apcotex industries ltd,technocraft industries india ltd,0
2,rishichem distributors pvt ltd,dsa,0
3,powermax rubber factory,co one,0
4,tress,longyou industries park zhejiang,0
...,...,...,...
497814,bitmat products,goodyear tire rubber company,0
497815,bnd trading co ltd,zhong yue liang economy trade imp exp co ltd,0
497816,xeikon industrial co ltd dongguan city,yi cheng trading co ltd dongguan city,0
497817,shanghai kechuan trading co ltd,shanghai mg stationery inc,0


In [12]:
data = (cleaned_data['name_1'] + cleaned_data['name_2']).unique()
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [16]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count, epochs=1)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [7]:
# cleaned_data['encoded_name_2'] = cleaned_data['name_2'].apply(lambda x: tokenizer(str(x), return_tensors='pt', padding='max_length', max_length=8)['input_ids'][0].numpy())

In [8]:
# cleaned_data['concated'] = cleaned_data.apply(lambda row: np.concatenate((row['encoded_name_1'], row['encoded_name_2']), axis=None), axis=1)

In [18]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)



V1_infer [-0.00414601  0.00348422 -0.00630662  0.00318817  0.00493348 -0.02080693
  0.00079859  0.02112808  0.00177588 -0.0278485   0.0114997  -0.01934132
  0.02478663 -0.01403242 -0.01967605 -0.00321959  0.00151979 -0.0011869
  0.00478925 -0.02962819]


In [19]:
dublicates = cleaned_data.loc[cleaned_data['is_duplicate']==1]

In [20]:
dublicates = dublicates.reset_index()

In [29]:
a = model.infer_vector(word_tokenize(dublicates['name_1'][456]))
b = model.infer_vector(word_tokenize(dublicates['name_2'][456]))


In [30]:
print(1 - spatial.distance.cosine(model.infer_vector(word_tokenize(cleaned_data['name_1'][46])),model.infer_vector(word_tokenize(cleaned_data['name_2'][46]))))
print(1 - spatial.distance.cosine(a,b))

-0.014698543585836887
0.11145703494548798
